## TrustPilot reviews

We'll be performing some topic classification using TrustPilot reviews.

In [1]:
import fetch
from fetch.utils import json_or_fetch

In [2]:
url = 'https://dk.trustpilot.com/review/www.netcompany.com'

In [3]:
reviews = json_or_fetch(url, fetch.trustpilot, 'data/trustpilot.json')